## BloomTech Data Science

---


# Cross-Validation

- Do **k-fold cross-validation** with independent test set
- Use scikit-learn for **hyperparameter optimization**

In [ ]:
%%capture
!pip install category_encoders==2.*

In [ ]:
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, validation_curve # k-fold CV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV # Hyperparameter tuning
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Downloading the Tanzania Waterpump Dataset

Make sure  you only use the dataset that is available through the **DS** **Kaggle Competition**. DO NOT USE any other Tanzania waterpump datasets that you might find online.

There are two ways you can get the dataset. Make sure you have joined the competition first!:

1. You can download the dataset directly by accessing the challenge and the files through the Kaggle Competition URL on Canvas (make sure you have joined the competition!)

2. Use the Kaggle API using the code in the following cells. This article provides helpful information on how to fetch your Kaggle Dataset into Google Colab using the Kaggle API.

> https://medium.com/analytics-vidhya/how-to-fetch-kaggle-datasets-into-google-colab-ea682569851a

# Using Kaggle API to download datset

In [ ]:
# mounting your google drive on colab
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#change your working directory, if you want to or have already saved your kaggle dataset on google drive.
%cd /content/gdrive/My Drive/Kaggle
# update it to your folder location on drive that contians the dataset and/or kaggle API token json file.

/content/gdrive/My Drive/Kaggle


In [ ]:
# Download your Kaggle Dataset, if you haven't already done so.
# import os
# os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# !kaggle competitions download -c bloomtech-water-pump-challenge

In [ ]:
# Unzip your Kaggle dataset, if you haven't already done so.
# !unzip \*.zip  && rm *.zip

In [ ]:
# List all files in your Kaggle folder on your google drive.
!ls

archive.zip  kaggle.json	     sample_submission.csv  train_features.csv	winequality-red.csv
drive	     red-wine-quality.ipynb  test_features.csv	    train_labels.csv


# I. Wrangle Data

In [ ]:
def wrangle(fm_path, tv_path=None):
  if tv_path:
    df = pd.merge(pd.read_csv(fm_path,
                              na_values=[0, -2.000000e-08],
                              parse_dates=['date_recorded']),
                  pd.read_csv(tv_path)).set_index('id')
  else:
    df = pd.read_csv(fm_path,
                     na_values=[0, -2.000000e-08],
                     parse_dates=['date_recorded'],
                     index_col='id')

  # Drop constant columns
  df.drop(columns=['recorded_by'], inplace=True)

  # Create age feature
  df['pump_age'] = df['date_recorded'].dt.year - df['construction_year']
  df.drop(columns='date_recorded', inplace=True)

  # Drop HCCCs
  cutoff = 100
  drop_cols = [col for col in df.select_dtypes('object').columns
              if df[col].nunique() > cutoff]
  df.drop(columns=drop_cols, inplace=True)

  # Drop duplicate columns
  dupe_cols = [col for col in df.head(100).T.duplicated().index # change 15 to 100!!!!
               if df.head(100).T.duplicated()[col]]
  df.drop(columns=dupe_cols, inplace=True)

  return df

df = wrangle(fm_path='train_features.csv',
             tv_path='train_labels.csv')

X_test = wrangle(fm_path='test_features.csv')

In [ ]:
df.shape, X_test.shape

((47519, 31), (11880, 30))

# II. Split Data

## Split TV from FM

In [ ]:
target = 'status_group'
y = df[target]
X = df.drop(columns = target)

# III. Establish Baseline

This is a **classification** problem, our baseline will be **accuracy**.

In [ ]:
y.value_counts(normalize = True).max()

0.5429828068772491

# IV. Build Models

- `DecisionTreeClassifier`
- `RandomForestClassifier`

In [ ]:
model_dt = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    DecisionTreeClassifier(random_state=42))

In [ ]:
model_rf = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(n_estimators =25, random_state=42)
);

**Check cross-validation scores**

![Cross Validation](https://upload.wikimedia.org/wikipedia/commons/4/4b/KfoldCV.gif)

In [ ]:
cv_score_dt = cross_val_score(model_dt, X, y, cv = 5, n_jobs = -1)
cv_score_rf = cross_val_score(model_rf, X, y, cv = 5, n_jobs = -1)

In [ ]:
cv_score_dt

array([0.74400253, 0.75210438, 0.74915825, 0.74957912, 0.7465011 ])

In [ ]:
cv_score_rf

array([0.79282407, 0.79703283, 0.79819024, 0.7996633 , 0.79385457])

In [ ]:
print('CV score DecisionTreeClassifier')
print('Mean CV accuracy score:', cv_score_dt.mean())
print('STD CV accuracy score:',cv_score_dt.std())

CV score DecisionTreeClassifier
Mean CV accuracy score: 0.7482690762017027
STD CV accuracy score: 0.002776529048546816


In [ ]:
print('CV score RandomForestClassifier')
print('Mean CV accuracy score:',cv_score_rf.mean())
print('STD CV accuracy score:',cv_score_rf.std())

CV score RandomForestClassifier
Mean CV accuracy score: 0.7963130019901566
STD CV accuracy score: 0.0025877845966532738


# V. Tune Model

Different hyperparameter values I want to try out :

*   Simpleimputer - mean, median - 2 values
*   max_depth - range(5,40,5) - 7 values
*   n_estimators - range(25,125,25) - 4 values

> Total combinations of these hyperparameters = 2 * 7 * 4 = 56

Testing out the above hyperparameter combinations with 5-fold Cross Validation will need :

> Total number of models to be fit = 2 * 7 * 4 * 5 = 280


In [ ]:
clf = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state = 42,n_jobs = -1)
)

In [ ]:
param_grid = {
    'simpleimputer__strategy':['mean', 'median'],
    'randomforestclassifier__max_depth':range(5,40,5),
    'randomforestclassifier__n_estimators':range(25,125,25)
}

**`GridSearch`:**

In [ ]:
model_rf_gs = GridSearchCV(clf,
            param_grid = param_grid,
             n_jobs = -1,
             cv = 3,
            verbose = 1,
            )
model_rf_gs.fit(X,y)

Fitting 3 folds for each of 56 candidates, totalling 168 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('ordinalencoder', OrdinalEncoder()),
                                       ('simpleimputer', SimpleImputer()),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(n_jobs=-1,
                                                               random_state=42))]),
             n_jobs=-1,
             param_grid={'randomforestclassifier__max_depth': range(5, 40, 5),
                         'randomforestclassifier__n_estimators': range(25, 125, 25),
                         'simpleimputer__strategy': ['mean', 'median']},
             verbose=1)

In [ ]:
model_rf_gs.best_params_

{'randomforestclassifier__max_depth': 20,
 'randomforestclassifier__n_estimators': 100,
 'simpleimputer__strategy': 'median'}

In [ ]:
model_rf_gs.best_score_

0.7997222830723208

In [ ]:
model_rf_gs.best_estimator_

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['basin', 'region', 'public_meeting',
                                      'scheme_management', 'permit',
                                      'extraction_type',
                                      'extraction_type_group',
                                      'extraction_type_class', 'management',
                                      'management_group', 'payment',
                                      'payment_type', 'water_quality',
                                      'quality_group', 'quantity', 'source',
                                      'source_type', 'source_class',
                                      'waterpoint_type',
                                      'waterpoin...
communal standpipe multiple    4
improved spring                5
cattle trough                  6
dam                            7
NaN                           -2
dtype: int64},
                                         {'col': 'waterpoint_type_group',
                                          'data_type': dtype('O'),
                                          'mapping': communal standpipe    1
hand pump             2
other                 3
improved spring       4
cattle trough         5
dam                   6
NaN                  -2
dtype: int64}])),
                ('simpleimputer', SimpleImputer(strategy='median')),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=20, n_jobs=-1,
                                        random_state=42))])

In [ ]:
model_rf_gs.predict(X_test)

array(['non functional', 'functional', 'functional', ...,
       'non functional', 'functional', 'functional'], dtype=object)

**`RandomizedSearchCV`:**

In [ ]:
model_rf_rs = RandomizedSearchCV(clf,
            param_grid = param_grid,
             n_jobs = -1,
             cv = 3,
            verbose = 1,
            )
model_rf_rs.fit(X,y)

TypeError: RandomizedSearchCV.__init__() got an unexpected keyword argument 'param_grid'

In [ ]:
model_rf_rs = RandomizedSearchCV(
    clf,
    param_distributions= param_grid,
    n_jobs = -1,
    verbose = 1,
    n_iter= 7,
    cv = 5
)
model_rf_rs.fit(X,y)

In [ ]:
model_rf_rs.best_params_

In [ ]:
model_rf_rs.best_score_

In [ ]:
model_rf_rs.best_estimator_

# VI. Communicate Results

**Showing Feature Importance**

Plot the feature importance for our `RandomForest` model.

In [ ]:
bestestimator = model_rf_rs.best_estimator_
importances = bestestimator.named_steps['randomforestclassifier'].feature_importances_
features = X.columns
feat_imp = pd.Series(importances, index=features).sort_values()
feat_imp.tail(10).plot(kind='barh')
plt.xlabel('Reduction in Gini Impurity');

# VII. Make Submission

In [ ]:
y_pred = model_rf_rs.predict(X_test)

In [ ]:
submission = pd.DataFrame({'status_group':y_pred}, index=X_test.index)

In [ ]:
submission

In [ ]:
pd.Timestamp.now().strftime('%Y-%m-%d_%H%M_')

In [ ]:
datestamp = pd.Timestamp.now().strftime('%Y-%m-%d_%H%M_') #string from time format
submission.to_csv(f'{datestamp}submission.csv') #format string

# VIII. Saving a trained model to reuse it in the future

In [ ]:
# Once you have found the best model, you might as well save it and then reload it when you want to test it later

# save model
import pickle

filename =

#save your model (it will be stored in your current working directory - download to your computer if GDrive is not mounted)
pickle.dump(model_rf,open(filename,'wb'))
#load model
model_rf_loaded = pickle.load(open(filename,'rb'))